In [2]:
!pip install music21

  Obtaining dependency information for music21 from https://files.pythonhosted.org/packages/a1/1b/ef7bdf01df19cc7ac9294531a9c991c6d382bde6bc15c9d106b9a5e547ef/music21-9.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 4.3 MB/s eta 0:00:00


In [3]:
import glob
import pickle
import pandas as pd
import numpy
import os
from sklearn.model_selection import train_test_split
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
import keras.utils
from keras.callbacks import ModelCheckpoint

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
notes_train = []
durations_train = [] 

notes_val = []
durations_val = [] 

In [7]:
from fractions import Fraction

notes = []
durations = []
with open("results_first/notes_train.txt", 'r') as notes_file:
    for line in notes_file:
        notes_train.append(line.strip())

with open("results_first//notes_val.txt", 'r') as notes_file:
    for line in notes_file:
        notes_val.append(line.strip())
        
        
with open("results_first/durations_train.txt", 'r') as durations_file:
    for line in durations_file:
        duration_str = line.strip()  # Get the duration string
        
        # Check if duration string is a fraction
        if '/' in duration_str:
            # Convert fraction string to float using the Fraction module
            fraction = Fraction(duration_str)
            durations_train.append(float(fraction))  # Convert Fraction to float
        else:
            # For other representations, try converting to float directly
            try:
                durations_train.append(float(duration_str))
            except ValueError:
                # If not convertible, keep as string
                durations_train.append(duration_str)
                
with open("results_first//durations_val.txt", 'r') as durations_file:
    for line in durations_file:
        duration_str = line.strip()  # Get the duration string
        
        # Check if duration string is a fraction
        if '/' in duration_str:
            # Convert fraction string to float using the Fraction module
            fraction = Fraction(duration_str)
            durations_val.append(float(fraction))  # Convert Fraction to float
        else:
            # For other representations, try converting to float directly
            try:
                durations_val.append(float(duration_str))
            except ValueError:
                # If not convertible, keep as string
                durations_val.append(duration_str)


In [8]:
len(notes_train), len(durations_train)

(287284, 287284)

In [9]:
len(notes_val), len(durations_val)

(75729, 75729)

In [10]:
sequence_length = 25

# Combine pitch and duration into tuples
combined_notes_durations = list(zip(notes_train, durations_train))

# get all unique combined pitch and duration pairs
unique_pairs = sorted(set(item for item in combined_notes_durations))

# create a dictionary to map unique pairs to integers
pair_to_int = {pair: number for number, pair in enumerate(unique_pairs)}
pair_to_int[("oov","oov")] = len(unique_pairs)

network_input = []
network_output = []

# create input sequences and corresponding outputs
for i in range(0, len(combined_notes_durations) - sequence_length, 1):
    sequence_in = combined_notes_durations[i:i + sequence_length]
    sequence_out = combined_notes_durations[i + sequence_length]

    # map pitches and durations to their integer representations
    network_input.append([pair_to_int[pair] for pair in sequence_in])
    network_output.append(pair_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(len(unique_pairs)+1)

# one-hot encode the output (since it represents both pitch and duration)
network_output = keras.utils.to_categorical(network_output, num_classes=len(unique_pairs)+1)

In [11]:
len(pair_to_int)

3017

In [12]:
sequence_length = 25

# Combine pitch and duration into tuples
combined_notes_durations_val = list(zip(notes_val, durations_val))

# get all unique combined pitch and duration pairs
unique_pairs_val = sorted(set(item for item in combined_notes_durations_val))

# create a dictionary to map unique pairs to integers
# pair_to_int = {pair: number for number, pair in enumerate(unique_pairs)}

network_input_val = []
network_output_val = []

# create input sequences and corresponding outputs
for i in range(0, len(combined_notes_durations_val) - sequence_length, 1):
    sequence_in_val = combined_notes_durations_val[i:i + sequence_length]
    sequence_out_val = combined_notes_durations_val[i + sequence_length]

    # map pitches and durations to their integer representations
    sequence_int_val = []
    for pair in sequence_in_val:
        # Check if the pair exists in pair_to_int, if not, use ("oov", "oov")
        if pair in pair_to_int:
            sequence_int_val.append(pair_to_int[pair])
        else:
            sequence_int_val.append(pair_to_int[("oov", "oov")])

    network_input_val.append(sequence_int_val)
#     network_input_val.append([pair_to_int[pair] for pair in sequence_in_val])
    try:
        network_output_val.append(pair_to_int[sequence_out_val])
    except:
        network_output_val.append(pair_to_int[("oov", "oov")])
#     network_output_val.append(pair_to_int[sequence_out_val])

n_patterns_val = len(network_input_val)

# reshape the input into a format compatible with LSTM layers
network_input_val = numpy.reshape(network_input_val, (n_patterns_val, sequence_length, 1))

# normalize input
network_input_val = network_input_val / float(len(unique_pairs)+1)

# one-hot encode the output (since it represents both pitch and duration)
network_output_val = keras.utils.to_categorical(network_output_val, num_classes=len(unique_pairs)+1)

In [15]:
n_vocab= len(unique_pairs)+1

from keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))

model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [16]:
filepath = "secondmodel-{epoch:02d}-{loss:.4f}-bigger.h5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
history = model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list, validation_data=(network_input_val, network_output_val))

Epoch 1/200
4489/4489 [==============================] - 131s 27ms/step - loss: 6.2539 - accuracy: 0.0172 - val_loss: 5.9926 - val_accuracy: 0.0169
Epoch 2/200
   1/4489 [..............................] - ETA: 1:48 - loss: 6.1879 - accuracy: 0.0156

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4489/4489 [==============================] - 118s 26ms/step - loss: 6.1134 - accuracy: 0.0290 - val_loss: 5.9450 - val_accuracy: 0.0225
Epoch 3/200
4489/4489 [==============================] - 119s 26ms/step - loss: 5.9723 - accuracy: 0.0429 - val_loss: 5.9507 - val_accuracy: 0.0279
Epoch 4/200
4489/4489 [==============================] - 118s 26ms/step - loss: 5.8538 - accuracy: 0.0545 - val_loss: 5.9224 - val_accuracy: 0.0312
Epoch 5/200
4489/4489 [==============================] - 118s 26ms/step - loss: 5.7291 - accuracy: 0.0652 - val_loss: 5.8767 - val_accuracy: 0.0349
Epoch 6/200
4489/4489 [==============================] - 125s 28ms/step - loss: 5.6323 - accuracy: 0.0756 - val_loss: 5.9064 - val_accuracy: 0.0334
Epoch 7/200
4489/4489 [==============================] - 118s 26ms/step - loss: 5.5315 - accuracy: 0.0869 - val_loss: 5.8893 - val_accuracy: 0.0362
Epoch 8/200
4489/4489 [==============================] - 120s 27ms/step - loss: 5.4279 - accuracy: 0.0993 - val_loss: 5.8584

In [17]:
import pickle

# Open a file and use dump() 
with open('results_second/history.pkl', 'wb') as file: 
    pickle.dump(history, file)
    
model.save("results_second/lstm.h5")
    
with open("results_second/pair_to_int.pkl", "wb") as file:
    pickle.dump(pair_to_int, file)

In [18]:
from music21 import note, stream, duration, midi

# Create a reverse dictionary to map integers back to combined note-duration pairs
int_to_pair = {number: pair for pair, number in pair_to_int.items()}

start = numpy.random.randint(0, len(network_input)-1)
pattern = network_input[start]

prediction_output = []

# generate notes
for note_index in range(100):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    
    result = int_to_pair[index]  # Retrieve the note-duration pair from the integer index
    prediction_output.append(result)
    
    # Update pattern for the next iteration
    to_append = index / float(len(unique_pairs))
    pattern = numpy.append(pattern, [[to_append]], axis=0)
    pattern = pattern[1:len(pattern)]

In [19]:
prediction_output

[('E-4', 0.5),
 ('E4', 0.5),
 ('G#4', 0.5),
 ('G4', 0.5),
 ('G#4', 0.5),
 ('C#4', 0.5),
 ('C#4', 0.5),
 ('E-5', 0.5),
 ('E-4', 0.5),
 ('E4', 0.5),
 ('F3', 0.5),
 ('G4', 0.5),
 ('C4', 0.5),
 ('C4', 0.5),
 ('F3', 0.25),
 ('F5', 0.5),
 ('G4', 0.25),
 ('G#4', 0.5),
 ('C#4', 0.5),
 ('F3', 0.5),
 ('G#4', 0.5),
 ('G#4', 0.5),
 ('G#4', 0.5),
 ('G#4', 0.5),
 ('G#3', 0.5),
 ('G#4', 0.5),
 ('G#3', 0.5),
 ('G#3', 0.5),
 ('G#3', 0.5),
 ('C#4', 0.25),
 ('B-5', 0.25),
 ('B-5', 0.25),
 ('A4', 0.25),
 ('G#4', 1.0),
 ('B-4', 0.25),
 ('G4', 0.25),
 ('G4', 0.25),
 ('G#5', 0.5),
 ('F4', 0.5),
 ('F4', 1.0),
 ('F4', 1.0),
 ('F#4', 0.25),
 ('F4', 1.0),
 ('F4', 1.0),
 ('F4', 1.0),
 ('F4', 1.0),
 ('B-4', 1.0),
 ('B-4', 1.0),
 ('G#4', 0.5),
 ('G4', 0.25),
 ('G#4', 0.5),
 ('G#4', 0.5),
 ('F#4', 0.25),
 ('F4', 0.25),
 ('F#4', 0.25),
 ('F#4', 0.25),
 ('F#4', 0.25),
 ('F4', 1.0),
 ('F4', 1.0),
 ('D5', 1.0),
 ('D5', 1.0),
 ('F4', 1.0),
 ('F4', 0.5),
 ('F5', 1.0),
 ('F4', 0.5),
 ('F4', 0.5),
 ('F3', 0.5),
 ('F4', 0.5)

In [ ]:
output_stream = stream.Stream()

# Create note and duration objects and append them to output_notes
for pattern in prediction_output:
    note_pitch, note_duration = pattern

    # Check if the note_pitch is a float (representing a note duration) and handle accordingly
    if isinstance(note_pitch, float):
        # Assuming the previous note was a chord, end it and start a new note with a duration
        output_stream[-1].duration.quarterLength = note_pitch
    else:
        if '.' in note_pitch:  # If it's a chord
            notes_in_chord = note_pitch.split('.')
            chord_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.duration.quarterLength = note_duration
                chord_notes.append(new_note)
            new_chord = chord.Chord(chord_notes)
            output_stream.append(new_chord)
        else:  # If it's a single note
            new_note = note.Note(note_pitch)
            new_note.duration.quarterLength = note_duration
            output_stream.append(new_note)

# Write the MIDI file
midi_stream = midi.translate.music21ObjectToMidiFile(output_stream)
midi_stream.open('results_second/generated_music1.mid', 'wb')
midi_stream.write()
midi_stream.close()